In [1]:
from load_data import load_all_data
from utils import interaction_matrix
import numpy as np
import pandas as pd

import plotly.express as px
import sklearn.manifold as mf
import sklearn.decomposition as decomp

from collections import Counter

In [2]:
targets, users, interactions, tracks = load_all_data()

In [3]:

counts = pd.DataFrame.from_records(list(Counter(Counter(tracks.artist).values()).items()))

In [4]:
px.scatter(counts, x=0, y=1)

In [5]:
full_matrix = interaction_matrix(users, tracks, interactions, prob=False)

Constructing matrix: 100%|██████████| 213423/213423 [00:15<00:00, 13587.44it/s]


In [6]:
downprojected = mf.TSNE(n_components=2, perplexity=40).fit_transform(full_matrix)

In [26]:
def find_mean_age(users: pd.DataFrame, age: int, matrix: np.ndarray) -> np.ndarray:
    return np.mean([np.concatenate([user, [age]]) for i, user in enumerate(matrix)
                    if users.iloc[i].age == age], axis=0)

def mean_vecs(downprojected, users):
    return [find_mean_age(users, age, downprojected) for age in set(users.age)]

In [49]:
def find_mean_loc(users: pd.DataFrame, loc: str, matrix: np.ndarray) -> np.ndarray:
    return np.mean([user for i, user in enumerate(matrix) if users.iloc[i].nationality == loc], axis=0)

def mean_vecs_loc(downprojected, users):
    mean_locs = [find_mean_loc(users, age, downprojected) for age in set(users.nationality)]
    return mean_locs

In [8]:
len(downprojected)

13098

In [27]:
mean_age_vecs = mean_vecs(downprojected[:10_000], users)

/home/tmpr/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning:

Mean of empty slice.

/home/tmpr/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



In [28]:
mean_age_vecs

[array([10.53823551, 18.97381884,  0.        ]),
 array([-14.78862286,  26.18534851,   1.        ]),
 array([  3.88673941, -28.97059933,   2.        ]),
 array([-12.44313876,  21.73925781,   3.        ]),
 array([-23.5146788,   1.0804793,   4.       ]),
 array([ 13.55047226, -14.82376099,   5.        ]),
 array([ 12.07772636, -31.58803936,   6.        ]),
 array([32.07281876, 25.01763093,  7.        ]),
 nan,
 array([-7.87209177, -0.85249954, 11.        ]),
 array([-11.16736031,  25.75150847,  12.        ]),
 array([-17.03176777,  -2.93739621,  13.        ]),
 array([-18.21833297,  -1.79805019,  14.        ]),
 array([-13.4459205,  -5.7687952,  15.       ]),
 array([-6.19411739,  0.04608569, 16.        ]),
 array([-7.9713065 , -3.34203633, 17.        ]),
 array([-3.36344141, -3.09731888, 18.        ]),
 array([-6.39748911, -1.37784142, 19.        ]),
 array([-1.68376752,  1.61664216, 20.        ]),
 array([-1.30526757,  2.98171316, 21.        ]),
 array([-1.72939199,  1.03579887, 22.  

In [29]:
mean_age_vecs = np.array([row for row in mean_age_vecs if isinstance(row, np.ndarray)])

In [30]:
px.scatter(x=mean_age_vecs[:, 0], y=mean_age_vecs[:, 1], color=mean_age_vecs[:, 2])

In [50]:
mean_loc_vecs = mean_vecs_loc(downprojected[:10_000], users)

/home/tmpr/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning:

Mean of empty slice.

/home/tmpr/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



In [54]:
full = pd.DataFrame.from_records([(*row, loc) for row, loc in zip(mean_loc_vecs, set(users.nationality)) if isinstance(row, np.ndarray)])

In [55]:
full

,0,1,2
0,-3.389623,-4.302110,DK
1,38.135834,-10.609058,EH
2,20.287285,11.662827,MK
3,-38.321548,-32.540314,JO
4,-12.646892,-6.465706,ZA
...,...,...,...
131,-10.711391,14.998260,KG
132,-4.041378,13.213736,SJ
133,-2.239660,10.577775,TR
134,-25.829893,-7.817807,SA


In [56]:
px.scatter(full, x=0, y=1, color=2)

In [ ]:
px.histogram(users.nationality)

In [31]:
px.histogram(users.age)